# Title: Python Series – Day 40: Memory Management & Garbage Collection in Python

## 1. Introduction
**Memory Management** is the process by which applications read and write data. Python handles this automatically, allowing developers to focus on code rather than memory allocation.

**Key Concepts:**
- **Allocation:** Reserving memory for objects.
- **Deallocation:** Freeing memory when objects are no longer needed.
- **Garbage Collection (GC):** Automated process to reclaim unused memory.

## 2. Python Memory Model
Python uses two types of memory allocation:
1. **Stack Memory:** Stores static memory (function calls, references).
2. **Heap Memory:** Stores dynamic memory (objects, lists, dictionaries).

In Python, **everything is an object** stored in the private heap.

## 3. Reference Counting (Core Concept)
Python uses **Reference Counting** as its primary memory management mechanism.
- Every object tracks how many references point to it.
- When the count drops to **0**, the memory is immediately freed.

In [ ]:
import sys

a = [1, 2, 3]
# Reference count is usually higher than expected because getrefcount() creates a temporary reference
print(f"Initial Ref Count: {sys.getrefcount(a)}")

b = a # Increase count
print(f"After assigning to b: {sys.getrefcount(a)}")

del b # Decrease count
print(f"After deleting b: {sys.getrefcount(a)}")

## 4. Garbage Collection (GC)
Reference counting isn't enough (e.g., circular references). Python has a built-in **Garbage Collector** to handle complex cases.

In [ ]:
import gc

print(f"GC enabled: {gc.isenabled()}")
print(f"GC Thresholds (Gen0, Gen1, Gen2): {gc.get_threshold()}")

# Force a generic collection
gc.collect()

## 5. Generational Garbage Collection
Python's GC divides objects into 3 generations based on their lifespan:
- **Generation 0:** Newest objects. Collected frequently.
- **Generation 1:** Objects that survived Gen 0 collections.
- **Generation 2:** Oldest objects. Collected rarely.

In [ ]:
# View current counts of objects in each generation
print(f"Objects in (Gen0, Gen1, Gen2): {gc.get_count()}")

## 6. Circular References Issue
Reference counting fails when objects reference each other creating a cycle. The GC is needed to detect and clean this up.

In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

def create_cycle():
    node1 = Node(1)
    node2 = Node(2)
    
    # Creating cycle
    node1.next = node2
    node2.next = node1
    
    return "Cycle Created"

create_cycle()
# At this point, node1 and node2 have ref_count > 0 but are unreachable.
# GC will eventually clean them up.
print("Cycle created and function exited.")
print(f"Objects collected manually: {gc.collect()}")

## 7. Memory Leaks in Python
Common causes:
1. **Unclosed resources** (Files, DB connections).
2. **Global variables** that keep growing.
3. **Circular references** with `__del__` implemented (in older Python versions).

**Prevention:** Use Context Managers (`with` statement)!

## 8. Optimizing Memory Usage
1. **Generators:** Use `yield` instead of returning massive lists.
2. **__slots__:** Save memory in classes by preventing `__dict__` creation.

In [ ]:
import sys

class StandardPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class SlottedPoint:
    __slots__ = ('x', 'y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

p1 = StandardPoint(10, 20)
p2 = SlottedPoint(10, 20)

# Note: getsizeof only measures the object shell, not referenced attributes for standard classes
# However, slots generally reduce memory footprint of large numbers of instances.
print(f"Standard Dict Size: {sys.getsizeof(p1.__dict__)}")
try:
    print(p2.__dict__)
except AttributeError:
    print("SlottedPoint has no __dict__ (Saves memory!)")

## 9. Checking Memory Usage
Use `sys.getsizeof()` to check the size of an object in bytes.

In [ ]:
import sys

regular_list = [i for i in range(1000)]
generator_obj = (i for i in range(1000))

print(f"List Size: {sys.getsizeof(regular_list)} bytes")
print(f"Generator Size: {sys.getsizeof(generator_obj)} bytes")

## 10. Real-World Examples
Processing a large file without crashing RAM.

In [ ]:
# Simulating a large file read generator
def read_large_range(n):
    for i in range(n):
        yield f"Line {i}"

# Consumes very little memory
for line in read_large_range(5):
    print(line)

## 11. Practice Exercises
1. Create a class and verify its reference count increases when assigned to a list.
2. Use `gc.collect()` and check the return value when clearing circular references.
3. Compare the `sys.getsizeof` of a `tuple` vs a `list` with the same elements.
4. Create a class with `__slots__` containing 5 attributes.
5. Write a potentially memory-leaking function using global list appending and fix it.

## 12. Day 40 Summary
- **Memory Model:** Stack vs Heap.
- **Core Mechanism:** Reference Counting.
- **Cleanup:** Generational Garbage Collection.
- **Optimization:** `__slots__`, Generators, Context Managers.

**Next topic: Day 41 – Python Virtual Environments & Pip**